# EX13 Movie Recommendation  
  
## Objective  
  
1. CSR matrix가 정상적으로 만들어졌다.  
사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.  
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.  
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도를 측정하고 의미를 분석해보았다.

In [3]:
import pandas as pd

import numpy as np
import scipy
import implicit

1) 데이터 준비와 전처리

In [4]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


추천 시스템에서 3점 이하인 점수는 모델에 반영하는 것이 의미가 없을 것 같습니다  
3점 이상만 남겨줍니다  

In [5]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [6]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [7]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


전처리가 완료되었습니다  
MF model을 구상한 후, 내가 좋아할 만한 영화를 추천해보는 것이 고ㅏ제

분석하기 : ratings에 있는 유니크한 영화 개수

In [ ]:
# 13-3의 nunique를 참고!

ratings에 있는 유니크한 사용자 수

In [ ]:
# 13-3의 nunique를 참고!

가장 인기 있는 영화 30개(인기순)

In [ ]:
# 각 영화의 'rating'를 합친걸 가져와야겠습니다
# 복잡하지만 groupby, sort_values 등으로 해결할 수 있을듯
# 13-3의 #인기많은 아티스트 부분 참고

내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다

In [ ]:
# 13-3의 'zimin이 등장하는' 부분에 답이 있습니다

CSR matrix를 직접 만들어 봅시다

In [ ]:
# 13-6 참고하기

als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다  
이거 시간이 엄청 걸리는 부분이에요  

In [10]:
#모델 구축
from implicit.als import AlternatingLeastSquares

n_factors = 100              #유저와 아이템의 벡터를 몇 차원으로 할 것인지
n_regularization  = 0.01     #과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
n_iterations = 15            #epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [11]:
#모델 생성하고
als_model = AlternatingLeastSquares(factors=n_factors, regularization=n_regularization, use_gpu=False, 
                                    iterations=n_iterations, dtype=np.float32)

In [ ]:
#핏 = 트레인
als_model.fit(''' 여기에 csr matrix가 들어가게 됩니다 sparse matrix of type '<class 'numpy.int64'> ''')
#item행 x user열 로 입력을 받기 때문에, 반대의 경우라면 transpose 해주세요


내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요

In [ ]:
#13-7의 queen이 나오는 부분, 벡터 내적

내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다

In [ ]:
#13-8의 similar_items 

내가 가장 좋아할 만한 영화들을 추천받아 봅시다

In [ ]:
#13-8의 recommend 메소드 